In [ ]:
import torch
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import random
import torch
import src.wrappers.wrapers as wrapers
import json
from pathlib import Path

def load_model(model_path):
    path = Path(model_path)
    
    config = json.load(open(path.parent.parent/'config.json'))
    checkpoint = torch.load(path.as_uri, map_location=torch.device('cuda:0'))
    
    model = getattr(wrapers, config["MODEL_WRAPER"])(config)
    model.load_state_dict(checkpoint['state_dict'])

    return model, config

def predict(model, full_vertex, positional_encoding: bool):   
    pred = torch.zeros_like(full_vertex)
    for i in range(full_vertex.shape[0]):
        for j in range(full_vertex.shape[1]):
            r = random.randint(full_vertex.shape[2])
            pos = []
            
            if positional_encoding:
                pos = [i, j, r]

            dim1 = full_vertex[i, j, :]
            dim2 = full_vertex[i, :, r]
            dim3 = full_vertex[:, j, r]
            full_input = torch.tensor([*pos, *dim1, *dim2, *dim3], dtype=torch.float32)
            pred[i, j, :] = model(full_input.to("cuda:0"))
            del dim1, dim2, dim3, full_input
    return pred

def get_vertex_from_filepath(path: str):
    with h5py.File(path, 'r') as f:
        for name, data in f["V"].items():
            if name.startswith("step"):
                return data[()]

In [ ]:
model_path = ""
data_path = ""
model, config = load_model(model_path)
model.model.eval()
data = get_vertex_from_filepath(data_path)

prediction = predict(model, data, positional_encoding=config["positional_encoding"])